# TensorRider Algorithm
Building regression model with TensorFlow's native APIs.

# Load Images and Labels

In [6]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from matplotlib import pyplot as plt

TRAIN_FILE = 'train.tfrecords'
VALIDATION_FILE = 'val.tfrecords'

BATCH_SIZE = 256
EPOCHS = 300

def imgPreprocess(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    # tgt = 0.2989 * r + 0.5870 * g + 0.1140 * b
    tgt = 0.5 * r + 0.25 * g + 0.25 * b
    return tgt

def prepareDataArrays(iterator):
    X = np.zeros(4800)
    y = np.zeros(1)
    recordCounter = 0;

    for string_record in iterator:
        recordCounter += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        imageString = (example.features.feature['image'].bytes_list.value[0])
        label = (example.features.feature['label'].int64_list.value[0])

        image = np.fromstring(imageString, dtype=np.uint8)
        image = image.reshape((60, 80, 3))
        image = rgb2mono(image)
        image = image.reshape((4800))

        X = np.vstack((X,image))
        y = np.append(y,label)
        if recordCounter % 100 == 0:
            print(recordCounter,end = '->')

    y = y.reshape((recordCounter + 1,))
    y = y / 40
    return X, y

trainIterator = tf.python_io.tf_record_iterator(path="train.tfrecords")
valIterator = tf.python_io.tf_record_iterator(path="val.tfrecords")
testIterator = tf.python_io.tf_record_iterator(path="test.tfrecords")

# => Load saved data arrays

In [2]:
npRecall = np.load("arrays.npz")
X_train = npRecall["xtr"]
y_train = npRecall["ytr"]
X_val = npRecall["xval"]
y_val = npRecall["yval"]
X_test = npRecall["xt"]
y_test = npRecall["yt"]

# => Load new data from TFRecords *(SLOW)*

In [ ]:
fileName = ''
fileName = input('Filename = ')
fileName = fileName + '.npz'

print("\nTrain...")
X_train, y_train = prepareDataArrays(trainIterator)
print("\nVal...")
X_val, y_val = prepareDataArrays(valIterator)
print("\nTest...")
X_test, y_test = prepareDataArrays(testIterator)

np.savez(fileName, xtr = X_train, ytr = y_train, xval = X_val, yval = y_val, xt = X_test, yt = y_test)

# Prepare

In [7]:
def inputs(mode, batch_size, num_epochs):
    if mode == 'train':
        image = X_train
        label = y_train
    elif mode == 'test':
        image = X_test
        label = y_test
    elif mode == 'val':
        image = X_val
        label = y_val
    
    imagesBatch, labelsBatch = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=2,
        capacity=1000 + 3 * batch_size,
        min_after_dequeue=1000
    )
    
    return imagesBatch, labelsBatch

In [10]:
x,y = inputs('train', 256, 300)
print(x,y)

Tensor("shuffle_batch_3:0", shape=(256, 13595, 4800), dtype=float64) Tensor("shuffle_batch_3:1", shape=(256, 13595), dtype=float64)


# Define the Dense Layers

In [2]:
x = tf.placeholder(tf.float32, [None, 4800])
W = tf.Variable(tf.zeros([4800, 1]))
b = tf.Variable(tf.zeros([1]))
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.float32, [None, 1])

msqr = tf.reduce_mean(tf.losses.mean_squared_error(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(msqr)

# Train

In [ ]:
sess = tf.InteractiveSession()
for _ in range(EPOCHS):
    batch_xs, batch_ys = inputs(train=True, batch_size=BATCH_SIZE, num_epochs=EPOCHS)
    batch_xs, batch_ys = sess.run([batch_xs, batch_ys])
    _, error = sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})
    print(_,error)